In [1]:
using Revise
using Scattering
using LinearAlgebra
using Test

┌ Info: Precompiling Scattering [a5c306d0-0a40-4008-a012-bc604123031d]
└ @ Base loading.jl:1260


## Testing Particle module

### Testing Sphere

In [2]:
# Sphere constructor
s = Sphere(2.0, 1.0, 1.0, RVector([0.0, 0, 0]))

Sphere{Float64}(2.0, 33.510321638291124, 1.0, 1.0, [0.0, 0.0, 0.0])

In [3]:
# Sphere constructor using standard Vector
s = Sphere(2.0, 1.0, 1.0, [0.0, 0, 0])

Sphere{Float64}(2.0, 33.510321638291124, 1.0, 1.0, [0.0, 0.0, 0.0])

In [4]:
# test ArgCheck: Sphere.R > 0
Sphere(-1.0, 1.0, 1.0, [0, 0, 0])

DomainError: DomainError with R > 0 must hold. Got
R => -1.0:


In [5]:
# copy constructor
s1 = Sphere(s)
@test s == s1

Test Passed

In [6]:
# translate sphere to a new location
t = [1, 1, 1]
s1 = translate(s, t)
@test s.origin + t ≈ s1.origin

Test Passed

In [7]:
# rotate sphere
M = RotationMatrix(EulerAngle(π/2, π/2, π/2))
s1 = rotate(s, M)
@test s1 == s 

ErrorException: type Sphere has no field orientation

In [8]:
# testing length
@test length(s) == 1

Test Passed

In [9]:
# testing updatesize
updatesize(s, 3.0)

Sphere{Float64}(3.0, 113.09733552923255, 1.0, 1.0, [0.0, 0.0, 0.0])

In [10]:
@test getsize(s) == s.R

Test Passed

In [11]:
# DomainError expected
@test_broken getsize(s, 2)

Test Broken
  Expression: getsize(s, 2)

In [12]:
@test getsizearray(s) == [s.R]

Test Passed

### Testing Cylinder

In [13]:
w = [1.0, 2.0, 3.0]
normalize!(w)
a = [0, 1, 2]
u = cross(w, a)
normalize!(u)
v = cross(w, u)
normalize!(v)

@show u
@show v
@show w
[u v w] # P

u = [0.408248290463863, -0.816496580927726, 0.408248290463863]
v = [0.8728715609439696, 0.21821789023599242, -0.4364357804719848]
w = [0.2672612419124244, 0.5345224838248488, 0.8017837257372732]


3×3 Array{Float64,2}:
  0.408248   0.872872  0.267261
 -0.816497   0.218218  0.534522
  0.408248  -0.436436  0.801784

In [14]:
R = RotationMatrix(u, v, w)
R.R

3×3 StaticArrays.SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
 0.408248  -0.816497   0.408248
 0.872872   0.218218  -0.436436
 0.267261   0.534522   0.801784

In [15]:
# Cylinder constructor
c = Cylinder(10.0, 1.0, 1.0, 1.0, RVector([0, 0, 0.0]), R)

Cylinder{Float64}(10.0, 1.0, 314.1592653589793, 1.0, 1.0, [0.0, 0.0, 0.0], RotationMatrix{Float64}([0.408248290463863 -0.816496580927726 0.408248290463863; 0.8728715609439696 0.21821789023599242 -0.4364357804719848; 0.2672612419124244 0.5345224838248488 0.8017837257372732]))

In [16]:
# Cylinder constructor using a standard Vector
c = Cylinder(10.0, 1.0, 1.0, 1.0, [0, 0, 0.0], R)

Cylinder{Float64}(10.0, 1.0, 314.1592653589793, 1.0, 1.0, [0.0, 0.0, 0.0], RotationMatrix{Float64}([0.408248290463863 -0.816496580927726 0.408248290463863; 0.8728715609439696 0.21821789023599242 -0.4364357804719848; 0.2672612419124244 0.5345224838248488 0.8017837257372732]))

In [17]:
# test ArgCheck: Cylinder.R > 0
c = Cylinder(-10.0, 1.0, 1.0, 1.0, [0, 0, 0.0], R)

DomainError: DomainError with R > 0 must hold. Got
R => -10.0:


In [18]:
# test ArgCheck: Cylinder.L > 0
c = Cylinder(10.0, -1.0, 1.0, 1.0, [0, 0, 0.0], R)

DomainError: DomainError with L > 0 must hold. Got
L => -1.0:


In [19]:
# copy constructor
c1 = Cylinder(c)
@test c1 == c

Test Passed

In [20]:
# translate cylinder to a new location
t = [1, 1, 1]
c1 = translate(c, t)
@test c.origin + t ≈ c1.origin

Test Passed

In [21]:
# rotate a cylinder
M = RotationMatrix(EulerAngle(π/2, π/2, π/2))
c1 = rotate(c, M)

R1 = c.orientation
R2 = R1 * inv(M)
@test c1.orientation.R ≈ R2.R

Test Passed

In [22]:
# the axis along height direction in the original cylinder
w1 = R1.R[3,:]
@show w1
R2 = c1.orientation
# the axis along height direction in the rotated cylinder
w2 = R2.R[3,:]
@show w2
# Verify that rotation M actually rotate w1 to w2 after rotating the cylinder using rotate method.
@test w2 ≈ M*w1

w1 = [0.2672612419124244, 0.5345224838248488, 0.8017837257372732]
w2 = [-0.2672612419124244, 0.8017837257372732, 0.5345224838248488]


Test Passed

In [23]:
# Convert to axis angle representation, get the rotation angle for both particles
ag1 = EulerAxisAngle(c.orientation)
ag2 = EulerAxisAngle(c1.orientation)
rad2deg(ag1.θ), rad2deg(ag2.θ)

(77.63580469304388, 130.9257764336803)

In [24]:
# test length
@test length(c) == 2

Test Passed

In [25]:
# test updatesize
updatesize(c, 5.0)

Cylinder{Float64}(5.0, 1.0, 78.53981633974483, 1.0, 1.0, [0.0, 0.0, 0.0], RotationMatrix{Float64}([0.408248290463863 -0.816496580927726 0.408248290463863; 0.8728715609439696 0.21821789023599242 -0.4364357804719848; 0.2672612419124244 0.5345224838248488 0.8017837257372732]))

In [26]:
updatesize(c, 2.0, 2)

Cylinder{Float64}(10.0, 2.0, 628.3185307179587, 1.0, 1.0, [0.0, 0.0, 0.0], RotationMatrix{Float64}([0.408248290463863 -0.816496580927726 0.408248290463863; 0.8728715609439696 0.21821789023599242 -0.4364357804719848; 0.2672612419124244 0.5345224838248488 0.8017837257372732]))

In [27]:
# a DomainError expected
@test_broken updatesize(c, 3.0, 3)

Test Broken
  Expression: updatesize(c, 3.0, 3)

In [28]:
@test getsize(c) == c.R

Test Passed

In [29]:
@test getsize(c, 2) == c.L

Test Passed

In [30]:
# DomainError expected
@test_broken getsize(c, 3)

Test Broken
  Expression: getsize(c, 3)

In [31]:
# Note that it is a column vector
@test getsizearray(c) == [c.R, c.L]

Test Passed

### Test rotation operation

In [32]:
R1 = c.orientation.R

3×3 StaticArrays.SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
 0.408248  -0.816497   0.408248
 0.872872   0.218218  -0.436436
 0.267261   0.534522   0.801784

In [33]:
P1 = transpose(R1)

3×3 StaticArrays.SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
  0.408248   0.872872  0.267261
 -0.816497   0.218218  0.534522
  0.408248  -0.436436  0.801784

In [34]:
M = RotationMatrix(EulerAngle(π/3, π/4, π/5)).R

3×3 StaticArrays.SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
 -0.223006  -0.908443  0.353553
  0.789312   0.044565  0.612372
 -0.572061   0.415627  0.707107

In [35]:
P = transpose(M)

3×3 StaticArrays.SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
 -0.223006  0.789312  -0.572061
 -0.908443  0.044565   0.415627
  0.353553  0.612372   0.707107

In [36]:
R2 = R1*P

3×3 StaticArrays.SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
  0.795036  0.535848  -0.284226
 -0.547198  0.431432  -0.717246
 -0.261711  0.725764   0.636219

In [37]:
P2 = transpose(R2)

3×3 StaticArrays.SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
  0.795036  -0.547198  -0.261711
  0.535848   0.431432   0.725764
 -0.284226  -0.717246   0.636219

In [38]:
w1 = R1[3,:]

3-element StaticArrays.SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 0.2672612419124244
 0.5345224838248488
 0.8017837257372732

In [39]:
w2 = R2[3,:]

3-element StaticArrays.SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 -0.26171064377764547
  0.7257638476861679
  0.6362188116719432

In [40]:
@test M*w1 == w2

Test Passed